In [0]:
from pyspark.sql.types import StructType
from pyspark.sql.functions import current_timestamp, from_utc_timestamp


In [0]:
path_schema = '/Volumes/ingestao_dados/tabnews/datalake_tabnews/Ano_2025/Mês_02/Dia_02/Minuto_02/Segundo_10/*.json'
sample_df = spark.read.option('multiline', 'true').json(path_schema)

In [0]:
path = '/Volumes/ingestao_dados/tabnews/datalake_tabnews/Ano_2025/Mês_02/Dia_02/Minuto_02/Segundo_10/*.json'
df = (spark.read.option('multiline', 'true')
                .schema(sample_df.schema)
                .json(path))
df_com_data_requisicao = df.withColumn('data_da_requisicao', from_utc_timestamp(current_timestamp(), "America/Sao_Paulo"))
df_com_data_requisicao.createOrReplaceTempView('Raw_TabNews_2025')

In [0]:
%sql

SELECT * 
FROM Raw_TabNews_2025
QUALIFY ROW_NUMBER() OVER (PARTITION BY id ORDER BY UPDATED_AT DESC) = 1

In [0]:
(df_com_data_requisicao.write.mode('overwrite')
                       .format('delta')
                       .option('overwriteSchema', 'true')
                       .option('mergeSchema', 'true')
                       .saveAsTable('catalogo.bronze.Raw_TabNews_2025'))

In [0]:
%sql
CREATE TABLE IF NOT EXISTS catalogo.bronze.Raw_TabNews_2025
USING DELTA
AS SELECT * FROM Raw_TabNews_2025


In [0]:
spark.sql("DESCRIBE CATALOG catalogo").show()

In [0]:
spark.sql("CLEAR CACHE")
spark.sql("REFRESH TABLE catalogo.bronze.Raw_TabNews_2025")

In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS catalogo.bronze")

In [0]:
spark.sql("SHOW TABLES IN catalogo.bronze").show()

In [0]:
(df_com_data_requisicao.write
 .mode('overwrite')
 .format('delta')
 .option('overwriteSchema', 'true')
 .saveAsTable('catalogo.bronze.Raw_TabNews_2025'))

In [0]:
spark.sql("""
CREATE TABLE IF NOT EXISTS catalogo.bronze.Raw_TabNews_2025
USING DELTA
AS SELECT * FROM df_com_data_requisicao
""")